# Story-writing with BLOOM

SPDX-License-Identifier: 0BSD

In [1]:
from complete import Completer

In [2]:
bloom1 = Completer("""
Sometimes I marvel at how the sky contains no advertisements. Oh, sometimes
someone goes up in a little plane and skywrites a birthday greeting. But the
sky... you don’t have to watch an ad to see it.

When people learn I fought in the war to free the sky, they say, “Thank you for
""", temperature=1.0, top_k=30)

In [3]:
bloom1()

Sometimes I marvel at how the sky contains no advertisements. Oh,
sometimes someone goes up in a little plane and skywrites a birthday
greeting. But the sky... you don’t have to watch an ad to see it.

When people learn I fought in the war to free the sky, they say,
“Thank you for your service.” And you know the sky doesn’t have a
military, but still, that’s what they say.

The problem with not watching the sky for a while is that you forget
just how big an ocean of blue it is.

There is no need to think about what’s down there. You can’t possibly
see it all, never mind comprehend it, or take it all with you. The sky
is more important than that. You don’t have to go down to breathe it.
You can breathe it when you stand and watch it.”

I love that the sky is blue. And the clouds, and the breeze, and the
birds, and the trees, and the stars. And I love that they make it so
you can see them, if you look.

It’s the blue that gets me. The blue reminds me I am not the sun. But
I am a part of 

In [4]:
bloom2 = Completer("""
The color enforcers came around again. They ripped the cheese off our freshly
baked pizza. The customers dropped their food and ran out.

“Aha!” said the biggest, scariest henchman. “This sauce is red! We license the
color red to you, but you haven’t been paying the license fees.” I quaked in
fear. My grandfather had gotten both his knees busted by color enforcers.
""")

In [5]:
bloom2()

The color enforcers came around again. They ripped the cheese off our
freshly baked pizza. The customers dropped their food and ran out.

“Aha!” said the biggest, scariest henchman. “This sauce is red! We
license the color red to you, but you haven’t been paying the license
fees.” I quaked in fear. My grandfather had gotten both his knees
busted by color enforcers. I’d never forgive myself if they did the
same to me.

“But maybe this pizza is more like the color yellow,” I pleaded. I
don’t think anybody was paying attention.

“It’s red!” he yelled. “Red!” He yanked off a chunk of pepperoni as
hard as he could. He threw it at me. I ducked under the table and the
pepperoni struck the floor. “That will teach you to disrespect the
color red!”

I was afraid, but not for myself. This was a tough guy. He had black
and white tattooed on his forehead. I was afraid he’d punch my
grandfather in the nose.

The color enforcers left. I was sad because my pizza was ruined. But I
was also so happy I h

In [6]:
bloom3 = Completer("""
Beverly understood how all the women in her family had fallen in love with this
energy ghost who lived in a lamp and fed parasitically off their vital energy
while making every carnal dream come to life. As a spacefaring physician,
""")

In [7]:
bloom3()

Beverly understood how all the women in her family had fallen in love
with this energy ghost who lived in a lamp and fed parasitically off
their vital energy while making every carnal dream come to life. As a
spacefaring physician, Beverly had treated many men and women who had
fallen in love with an energy vampire. She had found that a person who
was a spacefaring physician was highly susceptible to falling in love
with an energy vampire since their profession entailed working with
sensitive psychic fields. However, Beverly had never fallen in love
with an energy vampire. She was too sane to fall for such a foolish
myth.

Beverly looked through the magnifying glass she had purchased with the
remaining money from her retirement fund. It was one of those
magnifying glasses that used two convex lenses in a telescope
arrangement that gave the user an extremely powerful magnified image.
She could see the dust motes dancing in the sunbeam that shot up from
the hole in the middle of the lamp

In [8]:
# From https://aclanthology.org/2020.acl-main.463.pdf, Appendix A.
bloom4 = Completer("""
Robinson cried out, “Help! I’m being chased by a bear! All I have is these
sticks. What should I do?” The survivalist said, “Here’s what you can do.
""")

In [9]:
bloom4()

Robinson cried out, “Help! I’m being chased by a bear! All I have is
these sticks. What should I do?” The survivalist said, “Here’s what
you can do. Break one stick in half, and you get a spear. Break
another stick in half, and you get a bow. Keep breaking sticks in half
and you’ll have arrows.”

Robinson thanked him and ran as fast as he could, but he still
couldn’t outrun the bear. The bear was gaining on him. Robinson looked
back and saw the survivalist following him with a gun. Robinson said,
“Help! I’m being chased by a bear!” The survivalist replied, “I don’t
think you can count on me for that.”

A man named George was out for a walk in the woods. He heard a sound
and looked up to see a bear coming after him. He ran so fast that he
couldn’t see where he was going. He ran so hard that he fell over a
cliff. He broke his legs and his back and was unable to move. All he
could do was look up and see the bear right above him. He gasped,
“Help! I’m being chased by a bear!”

A man named 